<a href="https://colab.research.google.com/github/melnikovknst/COOKIE_2024/blob/Furie/WindowFurie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
import librosa.display

def apply_stft_to_numpy_file(file_path, n_fft=512, hop_length=256):
    # Загружаем numpy файл (предположим, что это двумерный массив для 1 секунды с 4 каналами)
    data = np.load(file_path)  # Форма данных: (num_samples, num_channels)

    # Получаем число образцов и каналов
    num_samples, num_channels = data.shape

    # Рассчитываем число временных фреймов и частотных бинов
    num_time_frames = (num_samples - n_fft) // hop_length + 1
    num_frequency_bins = n_fft // 2 + 1

    # Создаем пустой массив для хранения результатов STFT
    stft_data = np.zeros((num_time_frames, num_frequency_bins, num_channels))

    # Применяем STFT для каждого канала
    for channel_idx in range(num_channels):
        channel_data = data[:, channel_idx]

        # Применяем STFT для канала (используем librosa для правильной работы с окнами и сдвигами)
        stft_result = librosa.stft(channel_data, n_fft=n_fft, hop_length=hop_length)

        # Берем амплитуду (модуль спектра) и сохраняем
        stft_data[:, :, channel_idx] = np.abs(stft_result.T)  # Транспонируем для согласования формы

    # Получаем имя файла без расширения
    base_name = os.path.splitext(os.path.basename(file_path))[0]

    # Формируем путь для сохранения нового numpy файла
    output_path = f"{base_name}_stft.npy"

    # Сохраняем данные в файл .npy
    np.save(output_path, {
        'num_samples': num_samples,
        'num_time_frames': num_time_frames,
        'num_frequency_bins': num_frequency_bins,
        'stft_data': stft_data
    })

    print(f"Данные сохранены в файл: {output_path}")

    # Визуализируем результаты STFT для каждого канала
    plt.figure(figsize=(15, 10))

    # Пройдемся по всем каналам и отобразим спектр для каждого
    for channel_idx in range(num_channels):
        plt.subplot(2, 2, channel_idx + 1)  # Создаем 2x2 подграфики для 4 каналов
        librosa.display.specshow(librosa.amplitude_to_db(stft_data[:, :, channel_idx], ref=np.max),
                                 aspect='auto', origin='lower', cmap='inferno')
        plt.colorbar(label='Амплитуда (дБ)')
        plt.title(f'STFT спектр для канала {channel_idx + 1}')
        plt.xlabel('Временные фреймы')
        plt.ylabel('Частотные бины')

    plt.tight_layout()
    plt.show()

# Пример использования:
file_path = '/content/drive/MyDrive/Colab Notebooks/2024_6_1 17_12_29.npy'  # Замените на путь к вашему numpy файлу

apply_stft_to_numpy_file(file_path)


ValueError: not enough values to unpack (expected 2, got 1)